In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [2]:
print(torch.cuda.device_count())            # Numero di GPU disponibili
print(torch.cuda.get_device_name(0))        # Nome della prima GPU disponibile
print(torch.cuda.current_device())        # Device in uso al momento
print(torch.cuda.set_device(0))             # Imposta la prima GPU come default
print(torch.cuda.get_device_capability(0))  # Verifica le capacità della prima GPU

1
Tesla T4
0
None
(7, 5)


In [3]:
path = './drive/MyDrive/Materiale_Pellegrino_personal/CIDDS_Meta/CIDDS_Meta.csv'
dataset = pd.read_csv(path)

### ***PRE-ELABORAZIONE DATI***

In [4]:
dataset

,Duration,Proto,Packets,Bytes,Flows,Flags,Tos,multilabel
0,0.245,TCP,2,670,1,.AP...,0,normal
1,0.000,TCP,1,66,1,.A....,0,normal
2,0.000,TCP,1,58,1,....S.,0,portScan
3,0.000,TCP,1,58,1,....S.,0,portScan
4,0.047,TCP,11,1027,1,.AP...,0,normal
...,...,...,...,...,...,...,...,...
399995,0.034,TCP,2,598,1,.AP...,0,normal
399996,0.000,TCP,1,95,1,.AP...,32,normal
399997,0.005,TCP,5,479,1,.AP.SF,0,dos
399998,0.000,TCP,1,66,1,.A...F,32,normal


In [10]:
print(Counter(dataset['Tos']))

Counter({0: 317233, 32: 82379, 192: 372, 16: 16})


In [ ]:
print('Multilabel datset: ', Counter(dataset['multilabel']))

Multilabel datset:  Counter({'normal': 243363, 'dos': 117904, 'portScan': 37723, 'pingScan': 646, 'bruteForce': 364})


In [ ]:
dataset = dataset.drop('Flows', axis=1)

In [ ]:
dep_var = 'multilabel'
cat_names = ["Proto", "Flags", 'Bytes']
cont_names = [col for col in dataset.columns if col not in cat_names and col != dep_var]

In [ ]:
print('Cont var: ', cont_names)
print('Cat var: ', cat_names)

Cont var:  ['Duration', 'Packets', 'Tos']
Cat var:  ['Proto', 'Flags', 'Bytes']


In [ ]:
# LabelEncoding della variabile target 
target_index = dataset.columns.get_loc(dep_var)
dataset.iloc[:, target_index] = LabelEncoder().fit_transform(dataset[dep_var])

#LabelEncoding delle variabili categoriali
for col in cat_names:
  target_index = dataset.columns.get_loc(col)
  dataset.iloc[:, target_index] = LabelEncoder().fit_transform(dataset[col])

In [ ]:
dataset

,Duration,Proto,Packets,Bytes,Flags,Tos,multilabel
0,0.245,2,2,11030,12,0,2
1,0.000,2,1,10946,4,0,2
2,0.000,2,1,10024,1,0,4
3,0.000,2,1,10024,1,0,4
4,0.047,2,11,233,12,0,2
...,...,...,...,...,...,...,...
399995,0.034,2,2,10228,12,0,2
399996,0.000,2,1,13159,12,32,2
399997,0.005,2,5,8690,15,0,1
399998,0.000,2,1,10946,5,32,2


In [ ]:
print('Encod multilabel dataset: ', Counter(dataset['multilabel']))

Encod multilabel dataset:  Counter({2: 243363, 1: 117904, 4: 37723, 3: 646, 0: 364})


In [ ]:
target_dict = {'bruteForce' : 0,
               'dos' : 1,
               'normal' : 2,
               'pingScan' : 3,
               'portScan' : 4}

In [ ]:
from sklearn.model_selection import train_test_split

# train 50% e test 50%
train, test = train_test_split(dataset, test_size=0.50)

In [ ]:
y_train = train[dep_var]
train = train.drop(dep_var, axis=1)
y_test = test[dep_var]
test = test.drop(dep_var, axis=1)

# validation di 2500 righe da train
train, validation, y_train, y_val = train_test_split(train, y_train, test_size=0.0125, random_state=0)

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
validation = validation.reset_index(drop=True)

y_test = y_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [ ]:
#### Fase di Categorical Embeddings ###############

for col in cat_names:
  train[col] = train[col].astype('category')

embedded_cols = {n: len(col.cat.categories) for n,col in train[cat_names].items()}
print(embedded_cols)

embedded_col_names = cat_names

# Determiniamo una funzione per la dimensione dell'incorporamento, presa da una libreria 
embedding_sizes = [(n_categories, min(200, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]
embedding_sizes

{'Proto': 4, 'Flags': 20, 'Bytes': 8934}


[(4, 2), (20, 10), (8934, 200)]

### ***MODEL***

In [ ]:
""" Pytorch Dataset e DataLoader
Estendiamo la Dataset classe (astratta) fornita da Pytorch per un accesso più facile al nostro set di dati durante l'addestramento 
e per utilizzare efficacemente  il DataLoader modulo per gestire i batch. Ciò comporta la sovrascrittura dei metodi __len__e __getitem__
secondo il nostro particolare set di dati.
Poiché abbiamo solo bisogno di incorporare colonne categoriali, dividiamo il nostro input in due parti: numerico e categoriale. """ 

class CIDDS_Dataset(Dataset):
    def __init__(self, X, Y, embedded_col_names):
        X = X.copy()
        self.X1 = X.loc[:,embedded_col_names].copy().values.astype(np.int64) #categorical columns
        self.X2 = X.drop(columns=embedded_col_names).copy().values.astype(np.float32) #numerical columns
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]
        
#creating train and valid datasets
train_ds = CIDDS_Dataset(train, y_train, embedded_col_names)
valid_ds = CIDDS_Dataset(validation, y_val, embedded_col_names)

In [ ]:
""" Making device (GPU/CPU) compatible

(borrowed from https://jovian.ml/aakashns/04-feedforward-nn)

In order to make use of a GPU if available, we'll have to move our data and model to it. """ 

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

device(type='cuda')

In [ ]:
""" I nostri dati sono suddivisi in parti continue e categoriali. Per prima cosa convertiamo le parti categoriali in vettori 
incorporanti in base alle dimensioni determinate in precedenza e le concateniamo con le parti continue per alimentare il resto della rete """ 

class CIDDS_Model(nn.Module):
    def __init__(self, embedding_sizes, n_cont):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings)  #length of all embeddings combined
        self.n_emb, self.n_cont = n_emb, n_cont
        self.lin1 = nn.Linear(self.n_emb + self.n_cont, 200)
        self.lin2 = nn.Linear(200, 70)
        self.lin3 = nn.Linear(70, 5)
        self.bn1 = nn.BatchNorm1d(self.n_cont)
        self.bn2 = nn.BatchNorm1d(200)
        self.bn3 = nn.BatchNorm1d(70)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
        

    def forward(self, x_cat, x_cont):
        #x = [e(x_cat[:,i]) for i,e in enumerate(self.embeddings)]
        x = [e(x_cat[:,0]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [ ]:
""" Fase di preparazione per l'addestramento """

# Optimizer
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

# Training function
def train_model(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x1, x2, y in train_dl:
        batch = y.shape[0]
        output = model(x1, x2)
        loss = F.cross_entropy(output, y)   
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

# Evaluation function
def val_loss(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x1, x2, y in valid_dl:
        current_batch_size = y.shape[0]
        out = model(x1, x2)
        loss = F.cross_entropy(out, y)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        pred = torch.max(out, 1)[1]
        correct += (pred == y).float().sum().item()
    #print("valid loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
    print('valid loss ', sum_loss/total, ' and accuracy ', correct/total)
    return sum_loss/total, correct/total

# Funzione per l'addestramento 
def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs): 
        loss = train_model(model, optim, train_dl)
        print('ep ', i, "training loss: ", loss)
        val_loss(model, valid_dl)

In [ ]:
model = CIDDS_Model(embedding_sizes, len(cont_names))
to_device(model, device)

CIDDS_Model(
  (embeddings): ModuleList(
    (0): Embedding(4, 2)
    (1): Embedding(20, 10)
    (2): Embedding(8934, 200)
  )
  (lin1): Linear(in_features=215, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=70, bias=True)
  (lin3): Linear(in_features=70, out_features=5, bias=True)
  (bn1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (emb_drop): Dropout(p=0.6, inplace=False)
  (drops): Dropout(p=0.3, inplace=False)
)

### ***TRAINING***

In [ ]:
""" Ora addestriamo il modello sul set di addestramento. Ho usato l'ottimizzatore Adam per ottimizzare la perdita di entropia incrociata. 
L'addestramento è piuttosto semplice: iterare attraverso ogni batch, eseguire un passaggio in avanti, calcolare i gradienti, 
eseguire una discesa del gradiente e ripetere questo processo per tutte le epoche necessarie. """ 

batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size,shuffle=True)

train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [ ]:
train_loop(model, epochs=500, lr=0.001, wd=0.02)

### ***PREDICTION***

In [ ]:
""" Effettuiamo le predizioni sul dataset di test """

test_ds = CIDDS_Dataset(test, np.zeros(len(test)), embedded_col_names)
test_dl = DataLoader(test_ds, batch_size=batch_size)
test_dl = DeviceDataLoader(test_dl, device)

# Utilizziamo la funzione softmax poiché siamo interessati alla probabilità per ogni classe
preds = []
with torch.no_grad():
    for x1,x2,y in test_dl:
        out = model(x1, x2)
        prob = F.softmax(out, dim=1)
        preds.append(prob)

In [ ]:
y_pred = []
for i in range(0, len(preds)):
  pred = preds[i].cpu()
  temp = np.argmax(pred, 1)
  temp = np.array(temp)
  y_pred = np.append(y_pred, temp)

y_pred = y_pred.astype(int)

In [ ]:
print('Test:', Counter(y_test))
print('Pred:', Counter(y_pred))

### ***EVALUATION***

In [ ]:
# Matrice di confusione, accuracy, classification_report
from sklearn.metrics import *

# y_test è la variabile che contiene i valori effettivi
# y_pred contiene i valori predetti dal modello
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
# non presente nella libreria, calcolo mediante formula
f2 = (1+2**2)*((precision*recall)/((2**2*precision)+recall))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(cm, target_dict)
disp.plot()

In [ ]:
mcm = multilabel_confusion_matrix(y_test, y_pred)
print(mcm)

In [ ]:
FP = cm.sum (axis = 0) - np.diag (cm) 
FN = cm.sum (axis = 1) - np.diag (cm) 
TP = np.diag (cm) 
TN = cm.sum () - (FP + FN + TP)

print('True positive: ', TP)
print('True negative: ', TN)
print('False positive: ', FP)
print('False negative: ', FN)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)

print('True positive rate: ', TPR)
print('True negative rate: ', TNR)
print('False positive rate: ', FPR)
print('False negative rate: ', FNR)

In [ ]:
print(cm)

In [ ]:
print(report)

In [ ]:
print('Accuracy: ', acc)
print('Precision_weighted: ', precision)
print('Recall_weighted: ', recall)
print('mcc: ', mcc)
print('f2: ', f2)